In [31]:
import ujson as json
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
results = 've-benchmarking/query_results/'

with open(results + "asr.json", "r") as f:
    results_asr = json.load(f)

with open(results + "caption.json", "r") as f:
    results_caption = json.load(f)

with open(results + "clip.json", "r") as f:
    results_clip = json.load(f)

with open(results + "ocr.json", "r") as f:
    results_ocr = json.load(f)

with open(results + "mix.json", "r") as f:
    results_mix = json.load(f)

In [61]:
def iou(query,candidate):
    # convert nanoseconds to seconds
    candidate_start = int(candidate["properties"]["start"]) / 1000000000.0
    candidate_end = int(candidate["properties"]["end"]) / 1000000000.0
    truth_start = query["segments"][0][0]
    truth_end = query["segments"][0][1]

    intersection = min(truth_end,candidate_end)-max(truth_start,candidate_start)
    if intersection < 0:
        # negative intersection = no overlap = IOU 0
        return 0.0
    
    union = (candidate_end-candidate_start) + (truth_end-truth_start) - intersection
    return intersection/union

In [62]:
def rank(query,candidates):
    truth = query["files"]
    rank = float("inf")
    for i in range(len(candidates)):
        cand_file = os.path.basename(candidates[i]["properties"]["path"])
        if cand_file in truth:
            rank = i+1
            break

    return rank

## Intersection-over-union for video retrievables

In [71]:
mix_iou = []
for query in results_mix:
    if "segments" in query["query"] and len(query["response"]["retrievables"])>0 and "start" in query["response"]["retrievables"][0]["properties"]: 
        iou_score = iou(query["query"],query["response"]["retrievables"][0])
        mix_iou.append({
            "query": query["query"],
            "iou": iou_score
        })


## MRR for image retrievables